In [1]:
from confluent_kafka import Consumer
import json
import time
from streamz import Stream
from dask.distributed import Client

In [2]:
import pandas as pd
from streamz.dataframe import DataFrame

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58819 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [5]:
def process(msg):
    val = msg[0]
    dic = json.loads(val.decode('utf-8'))
    df = pd.DataFrame([dic])
    mask = df['HEAD'] == 2
    df = df[mask]
    return df

In [6]:
def print_mean(val):
    print('Mean: ', val)
    
def print_sum(val):
    print('Sum: ', val)

In [7]:
input_topic = 'test'
consumer_config = { 
                    'bootstrap.servers': 'localhost:9092',
                    'group.id': 'mygroup',
                    'auto.offset.reset': 'latest'}

source = Stream.from_kafka_batched(input_topic, consumer_config, poll_interval='2s', dask = True)

In [8]:
stream_df = source.map(process)

In [10]:
example = pd.DataFrame({'HEAD':[], 'FPGA':[], 'TDC_CHANNEL':[], 'ORBIT_CNT':[], 'BX_COUNTER': [], 'TDC_MEAS': []})
sdf = DataFrame(stream_df, example = example)

In [11]:
#out_stream1 = sdf.tail(5).stream
out_stream2 = sdf.groupby('TDC_CHANNEL').count().HEAD.stream

In [12]:
out = out_stream2.gather()

#### EVOLVING_HEAD:

In [10]:
out_stream1

Output()

#### EVOLVING_TDC_CHANNEL:

In [14]:
#out_stream2

In [15]:
out

Output()

#### START STREAM:

In [16]:
source.start()

#### STOP STREAM:

In [13]:
source.stop()

#### STOP CLIENT:

In [ ]:
client.close()

#### TESTER_FILE:

In [ ]:
# file1: data_000001.txt
# file2: data_000637.txt

#### TEST_AREA:

In [124]:
mask = df['HEAD'] == 2
df = df[mask]

In [125]:
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,2,0,63,2252311494,3144,10.0
1,2,0,62,2252311494,3148,8.0
2,2,1,13,2252311502,2214,8.0
3,2,1,88,2252311509,945,5.0
4,2,1,116,2252311530,1582,29.0
...,...,...,...,...,...,...
1310710,2,0,118,2258818600,341,8.0
1310711,2,1,38,2258818600,342,1.0
1310712,2,0,119,2258818600,340,5.0
1310718,2,0,60,2258818609,3257,27.0


In [132]:
t = df.groupby('TDC_CHANNEL').count().HEAD

In [133]:
t

TDC_CHANNEL
0       9988
1       9647
2       5249
3       9578
4       9453
       ...  
125     9618
126    10421
127     9734
128      891
138     9519
Name: HEAD, Length: 130, dtype: int64